## Biterm Topic Model

### Import data

In [1]:
import json

with open('preprocessing/data.json', 'r') as f:
    documents = json.load(f)
    texts = [' '.join([word for word in text]) for text in documents]

In [2]:
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame(texts, columns=['headline_text'])
df.head()

,headline_text
0,апрель год коллаборация телескоп горизонт собы...
1,быстрый радиовсплеск год пора астрофизик удовл...
2,долгий время реакция циклоприсоединение реакци...
3,учёный чехия франция помощь метод трёхмерный д...
4,клетка друг друг помощь химический сигнал веще...


### Create corpus and vocabulary

In [3]:
from gensim import corpora

# Map each token to a unique ID
dictionary = corpora.Dictionary(documents)
print(f'Number of unique tokens: {len(dictionary)}')

# Filter out tokens by frequency
min_doc, max_doc = 15, .2
dictionary.filter_extremes(no_below=min_doc, no_above=max_doc)
print(f'Number of unique tokens (filtered): {len(dictionary)}')

# Create a BOW corpus
corpus = [dictionary.doc2bow(doc) for doc in documents]
print(f'Number of documents: {len(corpus)}')

Number of unique tokens: 43392
Number of unique tokens (filtered): 4901
Number of documents: 2289


### Create a sparse matrix and vocabilary from gensim's BOW document representation

In [4]:
import numpy as np
from gensim import matutils

def bow_iterator(docs, dictionary):
    for doc in docs:
        yield dictionary.doc2bow(doc)

def get_term_matrix(msgs, dictionary):
    bow = bow_iterator(msgs, dictionary)
    X = np.transpose(matutils.corpus2csc(bow).astype(np.int64))
    return X

def get_vocab(dictionary):
    tmp = []
    for i in range(len(dictionary)):
        tmp.append(dictionary.get(i))
    return tuple(tmp)

def get_vocab_dict(dictionary):
    return {k: v for k, v in dictionary.token2id.items()}


# Obtain term frequency in a sparse matrix and corpus vocabulary
# these are used instead of the default ones
X = get_term_matrix(documents, dictionary)
vocab = get_vocab(dictionary)
vocab_dict = get_vocab_dict(dictionary)

In [5]:
import bitermplus as btm

tf = np.array(X.sum(axis=0)).ravel()

# Vectorize documents
docs_vec = btm.get_vectorized_docs(texts, vocab)
docs_lens = list(map(len, docs_vec))

# Generate biterms
biterms = btm.get_biterms(docs_vec)

### Train and run the Biterm topic model

In [6]:
num_topics = 15
top_words = 15 # number of top words for coherence calculation

# Train the model
biterm_model = btm.BTM(X, vocab, seed=12321, T=num_topics, \
                       M=top_words, alpha=50/8, beta=0.01)

# Run the model
biterm_model.fit(biterms, iterations=50)

100%|██████████| 50/50 [01:01<00:00,  1.23s/it]


In [7]:
# Calculate documents vs topics probability matrix
p_zd = biterm_model.transform(docs_vec)

100%|██████████| 2289/2289 [00:00<00:00, 2958.90it/s]


### Show top words for each topic

In [8]:
num_words = 15
btm.get_top_topic_words(biterm_model, num_words)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14
0,галактика,физика,самка,растение,птица,кость,вымирание,волна,мутация,днк,химический,порода,неандерталец,мозг,позвоночный
1,масса,решение,самец,бактерия,насекомое,остаток,птица,электрон,хромосома,мышь,фермент,атмосфера,сапиенс,нейрон,нервный
2,звезда,наука,половой,эукариот,песня,находка,растение,поле,полезный,мутация,энергия,мантия,днк,сигнал,ветвь
3,частица,проект,потомство,насекомое,крыло,отложение,разнообразие,атом,приспособленность,опухоль,синтез,планета,неандертальский,мышь,червь
4,энергия,страна,пол,гриб,самец,морской,климат,энергия,аллель,вирус,бактерия,кора,орудие,рецептор,личинка
5,чёрный_дыра,участник,партнёр,паразит,динозавр,ископаемое,численность,частица,видообразование,бактерия,продукт,зона,человеческий,нервный,многоклеточный
6,ядро,язык,яйцо,муха,хищник,зуб,лес,квантовый,частота,экспрессия,атом,земной,культура,кора,билатерия
7,физика,специалист,размножение,водоросль,самка,порода,морской,излучение,адаптация,белка,мембрана,глубина,кость,обучение,губка
8,нейтрино,ребёнок,птица,муравей,семейство,палеонтолог,остров,частота,вредный,рак,соединение,океан,африка,социальный,рыба
9,излучение,книга,репродуктивный,хозяин,окраска,рыба,хищник,магнитный_поле,изменчивость,ткань,ион,кратер,денисовец,память,конечность


### Calculate metrics

In [13]:
perplexity = btm.perplexity(biterm_model.matrix_topics_words_, p_zd, X, num_topics)
coherence = btm.coherence(biterm_model.matrix_topics_words_, X, M=top_words)

print(f'Perplexity: {perplexity}\n' +
      f'Average coherence score: {sum(coherence)/num_topics}')

Perplexity: 1216.4758185708436
Average coherence score: -156.520366779074


### Show most probable topic for each document

In [10]:
btm.get_docs_top_topic(texts, biterm_model.matrix_docs_topics_)

,documents,label
0,апрель год коллаборация телескоп горизонт собы...,0
1,быстрый радиовсплеск год пора астрофизик удовл...,0
2,долгий время реакция циклоприсоединение реакци...,10
3,учёный чехия франция помощь метод трёхмерный д...,7
4,клетка друг друг помощь химический сигнал веще...,13
...,...,...
2284,головоногий моллюск наутилус глаз похожий слож...,14
2285,мезомицетозой открытый одноклеточный родственн...,14
2286,современный биология исследование ген семейств...,14
2287,бесчелюстной древний группа позвоночный животн...,14


### Show top documents for each topic

In [11]:
docs_num = 5
btm.get_top_topic_docs(texts, biterm_model.matrix_docs_topics_, docs_num)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14
0,чёрный_дыра интересный объект космолог астрофи...,жизнь дмитрий борисович зимин горестный отчаян...,конфликт эволюционный интерес самец самка секс...,фенотипический признак насекомое ген насекомое...,палеонтолог бирманский меловой янтарь возраст ...,группа американский учёный раннемеловой шляпоч...,группа американский учёный последний происхожд...,новый явление среда перепрыгивание фонон твёрд...,инверсия хромосомный перестройка участок хромо...,перспективный подход лечение рак применение т-...,металло-карбеноид необычный нестабильный молек...,результат датирование образец лава плато керге...,коллектив ведущий генетик палеоантрополог вели...,чувство радость удовольствие активность дофами...,жаберный щель уникальный признак тип хордовое ...
1,главный коллаборация большой адронный коллайде...,жизнь дмитрий борисович зимин горестный отчаян...,животное оптимальный репродуктивный стратегия ...,эволюционный связь насекомое-опылитель цветков...,пернатый ископаемый мезозойский эра очередной ...,вендобионт загадочный гигантский ископаемое ор...,сильный сокращение поверхность арктический лёд...,японский учёный возможность использование свер...,цихлида африка небольшой вулканический озёры к...,терапия помощь лимфоцит собственный иммунный к...,немецкий химик жидкометаллический катализатор ...,результат петролый-термомеханический моделиров...,достижение палеогеномик предок современный чел...,человек животное безобидный вещь прошлый болез...,проблема происхождение уникальный трубчатый це...
2,галактика внешний среда газ ядро сверхмассивны...,октябрь лауреат нобелевский_премия экономика г...,самец насекомое паук время ухаживание самка св...,американский муравей-листорез муравейник грибн...,американский учёный глубокий чёрный окраска пе...,палеонтолог южный_америка новый крупный диноза...,спор причина массовый вымирание крупный животн...,сверхпроводимость квантовый состояние вещество...,сравнительный анализ полный геном африканский ...,американский учёный роль корень рак раковый ст...,хлорoсилан важный реагент химический промышлен...,начало мезозой суперконтинент пангей часть зон...,команда сванте пеэбый работа геном древний чел...,крыса друг друг безопасный еда навык связанный...,hox-ген важный роль передне-задний разметка те...
3,группа учёный специальный_астрофизический_обсе...,социолог результат пилотный исследование малои...,десятилетний наблюдение популяция вилорог резе...,швейцарский учёный дивергентный эволюция репа ...,янтарь возраст год птенец примитивный энанциор...,год начало кембрийский период земля бурный раз...,район акватория южный_океан оконечность антарк...,кажущийся простота трение дискуссионный пробле...,важный этап видообразование формирование генет...,немецкий учёный новый противораковый вакцина м...,группа канадский учёный новый механизм нейтрал...,моделирование суперкомпьютер интерпретация сей...,международный команда палеогенетик глава свант...,трансгенный мышь аналог человеческий болезнь а...,международный команда учёный изучение интересн...
4,международный группа астрофизик далёкий массив...,историк возможный связь усложнение общественны...,рыба-игла морской коньковый забота потомство м...,сложный симбиотический отношение живой организ...,птица известный разнообразный голос пора палео...,известняк окрестность немецкий город зольнхофн...,дистанционный метод обследование поверхность з...,группа учёный констанцский_университет германи...,организм хромосомный перестройка генетический ...,месяц год успешный удаление раковый опухоль ра...,натрий-калиевый насос натрий-калиевый помпа бе...,океанический кора планета поверхностный слой п...,археологический остров юго-восточный_азия прих...,нейробиолог время бодрствование новый опыт неу...,время эмбриональный развитие зародыш зародышев...


### Visualize the results

In [16]:
import tmplot as tmp

phi = tmp.get_phi(biterm_model)
terms_probs = tmp.calc_terms_probs_ratio(phi, topic=5, lambda_=1)

tmp.plot_terms(terms_probs)

C:\Users\satan\AppData\Roaming\Python\Python39\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)